<a href="https://colab.research.google.com/github/khushboo-garg/SMCR/blob/main/Ques_1_2_Earth_Heat_2N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SMCR HW 2 _ Khushboo Garg

Ques 1:  Ask two different LLM’s :
 (a) to explain the code to you
 (b) if the code modeling the radiative and convective heat transfer correctly
 (c) if there anything missing in the heat transfer equations in the sys system function
 (d) what assumptions are implied by the code

(Chat GPT)
(a) This code builds a simple climate model of the Earth using layers of atmosphere and ocean.
It tries to show how heat energy moves between them over time.

1. The c class stores constants — numbers like the Sun’s heat, Earth’s size, and how good air/water are at holding and passing heat.
2. The sys function describes the physics — how heat moves between layers, how the Sun warms the ocean, and how heat escapes back to space.
3. The ode4u function is a math tool that steps through time to calculate how the system changes (like a slow-motion simulation).
4. The run() function sets up the starting temperatures, runs the simulation, and plots graphs of temperature and radiation change over time.

In short: the code is a small “Earth model” that calculates how the atmosphere and ocean exchange heat over 20 years.

(b) It models them in a simplified but reasonable way.

1. Radiative heat transfer (energy given off as light or infrared) is modeled using the Stefan–Boltzmann law (sbc * T**4) with emissivities (ee_atm, ee_ocn) for how much each layer emits or absorbs.
2. Convective (conductive) heat transfer (energy moving by contact between layers) is modeled with simple heat transfer coefficients (h_L, h_U) multiplied by temperature differences between neighboring layers.

These are basic but correct physical ideas — they represent how energy moves upward and downward.
However, the model is still very idealized and does not include all real-world complexity (like water vapor, clouds, or horizontal heat movement).

(c)Yes, a few things are simplified or missing:

1. No shortwave absorption in atmosphere — The sunlight only heats the ocean surface, not the atmosphere directly. In reality, some sunlight is absorbed by clouds, dust, and gases before reaching the surface.
2. No latent heat (evaporation and condensation) — Heat carried by water vapor (important for weather and climate) isn’t included.
3. No horizontal heat transport — It assumes the Earth is the same everywhere (no poles/equator difference).
4. No CO₂ or greenhouse gas dynamics — The greenhouse effect is fixed by constant emissivity values; it doesn’t change with gas concentration.
5. No mixing or advection — The ocean and air layers only transfer heat vertically, not sideways.

(d) The model assumes:

1. 1D (vertical only) — Heat only moves up and down, not sideways.
2. Steady properties — Densities, emissivities, and heat transfer rates don’t change with temperature or time.
3. Constant solar input — The sunlight (forcing) is steady, with no day/night or seasonal variation.
4. No weather or clouds — The air is clear, so no clouds or storms affect radiation.
5. Layered structure — The atmosphere and ocean are split into a few evenly spaced layers, each behaving uniformly.
6. Instant mixing within each layer — Every layer’s temperature is the same throughout that layer.
7. Fixed stratosphere temperature — The very top of the atmosphere is held constant at a set temperature.

(Claud)

(a) Code Explanation:
This code implements a multi-layer energy balance climate model that simulates heat transfer in Earth's atmosphere and ocean. The model uses:

1. An RK4 (4th-order Runge-Kutta) ODE solver to integrate heat equations over time
2. 8 atmospheric layers and 8 ocean layers representing vertical structure
3. Radiative heat transfer (longwave radiation) between layers using Stefan-Boltzmann law
4. Convective/conductive heat transfer between adjacent layers using temperature gradients
5. Solar forcing as the primary energy input at the surface

(b) Radiative and Convective Heat Transfer Assessment:
Radiative Transfer (in atmosphere): The code correctly implements:

1. Stefan-Boltzmann radiation: R ∝ T⁴
2. Layer emissivity (ε_atm = 0.78) for greenhouse effect
3. Transmission and absorption: each layer absorbs a fraction (ε_atm) and transmits the remainder (1-ε_atm)
4. Two-stream approximation: radiation flows both upward and downward

Convective Transfer: The code models this as simple diffusive heat flux:

1. Q̇ = h × ΔT, where h is the heat transfer coefficient and ΔT is temperature difference
2. Applied between adjacent atmospheric layers, ocean layers, and at the ocean-atmosphere interface

The physics is reasonably correct for a simplified climate model, though it makes several idealizations.

(c) Missing Elements in Heat Transfer Equations:
Several processes are absent or simplified:

1. Latent heat transport: No evaporation/condensation cycle (water vapor transport is a major heat redistribution mechanism)
2. Horizontal heat transport: The model is 1D (vertical only); no meridional or zonal heat transport
3. Cloud effects: No cloud radiative forcing (clouds significantly affect both shortwave and longwave radiation)
4. Shortwave absorption in atmosphere: Solar radiation goes directly to surface; no atmospheric absorption of incoming sunlight
5. Wind-driven mixing: Heat transfer coefficients are constant, not dependent on wind speed or stability
6. Ocean currents: No advective heat transport within the ocean

(d) Assumptions Implied by the Code:

1. One-dimensional (vertical only): Earth is treated as a single column with no horizontal variation - essentially modeling a globally-averaged climate
2. Constant layer properties: Each layer has uniform properties throughout its thickness
3. Gray atmosphere: Atmospheric emissivity is constant across all wavelengths (real atmosphere has wavelength-dependent absorption)
4. No phase changes: Water remains liquid/vapor without explicit modeling of condensation or freezing
5. Steady solar input: Solar constant is fixed (no diurnal or seasonal cycles)
6. Constant albedo: Surface reflectivity (α = 0.35) doesn't vary with ice cover, vegetation, or clouds
7. Exponential density profile: Atmospheric density follows a simple exponential decay with altitude
8. Fixed stratosphere temperature: Top boundary condition assumes constant stratospheric temperature (213 K)
9. Simple mixing parameterization: Heat transfer coefficients are prescribed constants rather than calculated from turbulence theory
10. No land-ocean distinction: The original code treats the entire surface as ocean (this was addressed in Question 3)
11. Radiative equilibrium in ocean: Ocean uses only conduction/convection, no internal radiative transfer
12. Time-invariant parameters: All physical constants remain fixed throughout the simulation

Despite these simplifications, the model captures the essential physics of vertical energy balance and provides useful insights into climate sensitivity to parameters like greenhouse gas concentrations (through emissivity) and mixing rates.


In [ ]:
import numpy as np , matplotlib.pyplot as plt

# Earth Heat 2N

In [ ]:
# ODE4U function
def ode4u(odefun, time, x0, u=None, c=None):
    """
    Solve a system of nonhomogeneous ODEs using the 4th-order Runge-Kutta method.
    (it depends on not just time and state but also external inputs (u) and constanstant (c))

    Parameters:
        odefun : function (t, x, u, c) -> (dxdt, y)
                 returns state derivative and output as arrays
        time   : time values at which the solution is computed.
                 p-dimensional array
        x0     : n-dimensional array, state at time[0].
        u      : (m x p) dimensional array
                 optional input sampled at each time step.
        c      : optional constants passed to odefun.

    Returns:
        time   : ndarray, shape (1, p)
        x_sol  : ndarray, shape (n, p)
        x_drv  : ndarray, shape (n, p)
        y_sol  : ndarray, shape (m, p)
    """

    time   = np.asarray(time)
    x0     = np.asarray(x0).flatten()
    points = len(time) # the total number of time steps

    # create defaults if not provided
    if c is None:
        c = 0
    if u is None:
        u = np.zeros((1, points))
    else:
        u = np.asarray(u)

    # verify inputs recieved are in 2D array shape
    if u.ndim == 1:
        u = u[np.newaxis, :]

    if u.shape[1] < points:
        pad_width = points - u.shape[1]
        u = np.pad(u, ((0, 0), (0, pad_width)), mode='constant')

    # state derivitives and outputs at time[0]
    dxdt0, y0 = odefun(time[0], x0, u[:, 0], c)

    n = x0.size                 # number of states
    m = np.asarray(y0).size     # nuber of outputs

    # allocate memory
    x_sol = np.ones([n, points])*np.nan
    x_drv = np.ones([n, points])*np.nan
    y_sol = np.ones([m, points])*np.nan

    x_sol[:, 0] = x0    # states
    x_drv[:, 0] = dxdt0 # state derivitives
    y_sol[:, 0] = y0    # outputs

    for p in range(points - 1):  # time stepping loop and main integration
        t = time[p]
        dt = time[p + 1] - t
        dt2 = dt / 2.0

        u_mid = (u[:, p] + u[:, p + 1]) / 2.0

        # intermediate dervitives
        dxdt1, _ = odefun(t + dt2, x0 + dxdt0 * dt2, u_mid, c)
        dxdt2, _ = odefun(t + dt2, x0 + dxdt1 * dt2, u_mid, c)
        dxdt3, _ = odefun(t + dt,  x0 + dxdt2 * dt, u[:, p + 1], c)

        # state update using the intermediate derivities
        x0 = x0 + ( dxdt0 + 2 * (dxdt1 + dxdt2) + dxdt3 ) * dt / 6.0

        # state derivitives and outputs at start of time (p+1)
        dxdt0, y0 = odefun(time[p + 1], x0, u[:, p + 1], c)

        # save states (solution), state derivitives, and outputs
        x_sol[:, p+1] = x0    # state
        x_drv[:, p+1] = dxdt1 # state derivitives
        y_sol[:, p+1] = y0    # output

        # safety - incase NaN or Inf
#       if not np.all(np.abs(x0) > 1e12):
        if not np.all(np.isfinite(x0)):
            break

    return time, x_sol, x_drv, y_sol

In [ ]:
#!/usr/bin/env python3

# 2N layer heat energy model for the earth's oceans and atmosphere

# https://en.wikipedia.org/wiki/Ocean_temperature
# https://en.wikipedia.org/wiki/Idealized_greenhouse_model
# https://en.wikipedia.org/wiki/Heat_transfer_coefficient
# https://www.engineersedge.com/thermodynamics/overall_heat_transfer-table.htm
# https://enghandbook.com/thermodynamics/heat-transfer/
# https://www.engineeringtoolbox.com/water-steam-thermal-diffusivity-d_2058.html

''' ... in console ...
import importlib , earth_heat_2N
importlib.reload(earth_heat_2N)     # to load edits
'''

# planetary constants
# -------------------
class c:
    pi  =   np.pi
    c   =  299.792458e6         # speed of light  m/s
    h   =    6.62607015e-34     # Planck's constant  J/Hz
    k   =    1.380649e-23       # Boltzmann's constant J/deg K
    sbc =    5.670367e-8        # Stefan-Boltzman constant  ... W/sq.m/deg K^4
    Ts  = 5777.                 # sun temperature, K
    Rs  =    6.9570e08          # radius of sun, m
    D   =    1.4960e11          # sun - to - earth distance, m
    Re  =    6.3567523e6        # radius of earth, m
    sec_per_yr = 365*24*3600    # seconds per year
    aa     =  0.35            #?# earth albedo 0.3 to 0.4 frctn of solar reflected
    ee_atm =  0.78            #?# atmos emissivity without GHG: 0.78 with GHG: 0.80
    ee_ocn =  0.96            #?# ocean emissivity 0.95 to 0.98
    years  = 20                 # years in the simulation
    n_layer = 8                 # layers in the ocean, layers in the atmosphere
#   RF = 4.00                 #?# radiative forcing imbalance  W/sq.m
#   lambda_LW = 4/3           #?# climate feedback parameter W/(sq.m K)
    T_strat = 273 - 60          # temperature of the stratosphere, K
    atm_density_s = 1.23        # density of air at sea level kg/cu.m
    atm_density_t = 0.32        # density of air at top of troposphere kg/cu.m
    atm_shc = 1003              # specific heat capacity of air J/(kg K)
    atm_heat_transfer =  3.7  #?# heat transfer rate of atm 5-30 W/(sq.m K)
    atm_elvtn = 12650            # depth of troposphere m
#   atm_water_cont= 0.004       # volume fraction of water in troposhere
    ocn_atm_heat_transfer=40  #?# heat transfer rate ocn-atm 5 to 50 W/(sq.m K)
    ocn_shc = 4000              # seawater specific heat capacity J/(kg K)
    ocn_density = 1023.6        # seawater density kg/cu.m
    ocn_heat_transfer =  4.0  #?# heat transfer rate of ocn 1-30 W / (sq.m K)
    ocn_depth = 3150            # average ocean depth m
    land_thickness = 8.4        # equivalent water thickness of land, m
    land_area_fraction = 0.292  # fraction of earth that is land . unitless

    # computed constants
    # ------------------
    solar_const = sbc*Ts**4.0 * 4*pi*Rs**2 / (4*pi*D**2.0) # solar constant W/sq.m

    # elevation of layers in the atm
    atm_z = atm_elvtn * np.hstack( [ 0 ,  np.logspace(-1.5,0.0,n_layer) ] )
    # elevation of layers in the ocn
    ocn_z = ocn_depth * np.hstack( [ 0 , -np.logspace(-1.5,0.0,n_layer) ] )

    atm_tz = atm_z[1:n_layer+1] - atm_z[0:n_layer]   # thickness of atm layer
    ocn_tz = ocn_z[0:n_layer]   - ocn_z[1:n_layer+1] # thickness of ocn layer

    atm_dz = 0.5*(atm_z[2:n_layer+1] - atm_z[0:n_layer-1]) # distance between atm layers
    ocn_dz = 0.5*(ocn_z[0:n_layer-1] - ocn_z[2:n_layer+1]) # distance between ocn layers
    ao__dz = 0.5*(atm_z[1] - ocn_z[1])                 # distance between atm&ocn layers

#   print("atm_z = " , np.round(atm_z))
#   print("ocn_z = " , np.round(ocn_z))
    print("ao__dz = ", np.round(ao__dz))

#   atmospheric density linear with altitude
#   atm_density = atm_density_s + (atm_density_t-atm_density_s)*atm_z/atm_elvtn
#   atmospheric density exponential with altitude
    atm_density_decay = atm_elvtn/np.log(atm_density_s / atm_density_t)
    atm_density = atm_density_s * np.exp(-(atm_z[0:n_layer]+0.5*atm_tz)/atm_density_decay)

#   print("atm_density", atm_density)

#   troposphere area heat capacity (per sq.m) J/(sq.m K)
    atm_ahc = atm_shc * atm_density * atm_tz
#   atm_ahc = ocn_shc * ocn_density * 50.0 # 50 m of water in 13000 m of atm
#   ocean       area heat capacity (per sq.m) J/(sq.m K)
    ocn_ahc = ocn_shc * ocn_density * ocn_tz


# define the dynamical system in terms of a set of o.d.e's
# --------------------------------------------------------
def sys(t, x, u, c):
    """
    compute the state derivives dxdt given current state, time, and input
    x = [ Q_atm , Q_ocn ] states ... heat in atmosphere and ocean J/sq.m
    """

    N  = c.n_layer
    # extract individual states from the vector of states , J/sq.m
    Q_atm = x[ 0 :   N ]       #   x( 0 :   N-1 )
    Q_ocn = x[ N : 2*N ]       #   x( N : 2*N-1 )

#   print("x= ", x)
#   print("Q_atm= ", Q_atm)
#   print("Q_ocn= ", Q_ocn)

    # convert from heat to temperature , K
    T_atm = np.maximum( 0.0 , Q_atm / c.atm_ahc )    # atmosphere deg K
    T_ocn = np.maximum( 0.0 , Q_ocn / c.ocn_ahc )    #   ocean    deg K

#   print("T_atm = ", T_atm)
#   print("T_ocn = ", T_ocn)

#   Atmosphere Heat .....................................
    dQ_atm_dt = np.ones(N)*np.nan

    R_out_U = c.ee_ocn * c.sbc * T_ocn[0]**4.0
    h_U = c.atm_heat_transfer

    for k in range(N):
        T_L = T_atm[k-1]
        h_L = c.atm_heat_transfer
        if k == 0:
            T_L = T_ocn[0]
            h_L = c.ocn_atm_heat_transfer
        if k == N-1:
            T_U = c.T_strat
        else:
            T_U = T_atm[k+1]

        # radiative inputs and outputs in an atmosphere layer
        # ---------------------------------------------------
        # radiant heat input through a layer's lower surface is
        # the radiant heat out of the lower layer's upper surface ...
        R_in_L  = R_out_U
        # radiant heat input through a layer's upper surface is
        # the radiant heat out of the upper layer's lower surface ...
        R_in_U  = c.ee_atm * c.sbc * T_U**4.0
        # radiant heat output through a layer's lower surface
        # depends on the layer's emissivity and the layer's temperature
        # and the radiant heat from transmitted through
        # the layer's upper surface that is not absorbed by the layer
        R_out_L = (1-c.ee_atm) * R_in_U  +  c.ee_atm * c.sbc * T_atm[k]**4.0
        # radiant heat output through a layer's upper surface
        # depends on the layer's emissivity and the layer's temperature
        # and the radiant heat from transmitted through
        # the layer's upper surface that is not absorbed by the layer
        R_out_U = (1-c.ee_atm) * R_in_L  +  c.ee_atm * c.sbc * T_atm[k]**4.0

        # rate of change in heat within atmosphere layer number "k"
        dQ_atm_dt[k] = R_in_L + R_in_U - R_out_L - R_out_U \
                       + h_L * (T_L - T_atm[k])  +  h_U * (T_U - T_atm[k])

#   Ocean Heat ......................................
    dQ_ocn_dt = np.ones(N)*np.nan

    h_L = c.ocn_heat_transfer

    for k in range(N):
        T_U = T_ocn[k-1]
        h_U = c.ocn_heat_transfer
        u0 = 0
        if k == 0:
            T_U = T_atm[0]
            h_U = c.ocn_atm_heat_transfer
            u0 = u[0]  -  c.ee_ocn*c.sbc*T_ocn[0]**4.0 \
                + (1-c.ee_atm)*c.sbc*T_atm[1]**4.0 \
                +     c.ee_atm*c.sbc*T_atm[0]**4.0
        if k == N-1:
            h_L = 0
            T_L = T_ocn[k]
        else:
            T_L = T_ocn[k+1]

        # rate of change in heat within ocean layer number "k"
        dQ_ocn_dt[k] = u0 + h_L * (T_L - T_ocn[k]) + h_U * (T_U - T_ocn[k])

#   RF Radiative Feedback ...........................
    RF = c.sbc * ( (1-c.ee_atm)*T_atm[1]**4.0 + c.ee_atm * T_atm[0]**4.0 \
                                              - c.ee_ocn * T_ocn[0]**4.0 )
#   print("RF = ", RF)

#   state derivitive ................................
    dxdt = np.hstack( [ dQ_atm_dt , dQ_ocn_dt ] ) # rate of change of heat per sq.m

#   output ..........................................
    y = np.hstack( [ T_atm , T_ocn , RF  ] )

    return dxdt, y


# simulate the dynamical system
# -----------------------------
def run():
    N = c.n_layer
    # time
    # --------------------
    time_end     = c.sec_per_yr  * c.years           # final time, s
    time_step    = c.sec_per_yr  / (12*365)          # time step, s
    n_time       = int( time_end / time_step )
    dt           = time_step * np.ones(n_time)
    n_ramp       = int( c.sec_per_yr * 9 / time_step )
    k_ramp       = np.linspace(0,n_ramp-1,n_ramp)
    dt[0:n_ramp] = time_step*( np.sin(2*np.pi*k_ramp/n_ramp/4) )**2.0
    time         = np.cumsum(dt);

    print("time_step=", time_step/3600, "hr ...  number of time steps=", n_time)
#   print("n_time_init=", n_time_init, "time_step_init=", time_step_init )

    # initial state values
    # --------------------
    # initial temp in the atmosphere and oceans deg.K
#   T_atm_init = np.linspace( 1 , -1 , N )
#   T_ocn_init = np.linspace( 1 ,  1 , N )
#   T_atm_init = np.linspace( 15 , -59 , N )
#   T_ocn_init = np.linspace( 17 ,   2 , N )
    T_atm_init = np.array( [ 15.,  -3.,  -17., -30., -40., -49., -55., -59.] )
    T_ocn_init = np.array( [ 19.,  17.,   14.,  12.,   9.,   7.,   4.,   3.] )

    print("T_atm_init = ", np.round( T_atm_init ) )
    print("T_ocn_init = ", np.round( T_ocn_init ) )

    x_init = np.hstack([ c.atm_ahc*(T_atm_init+273) , \
                         c.ocn_ahc*(T_ocn_init+273) ])

#   print("x_init = ", x_init)

    # external forcing
    # --------------------
    # short wavelength solar input over the surface W/sq.m
    u = (1-c.aa)*(c.solar_const/4)*np.ones(n_time)

#   print("u0 = ", u[0])

    # solve the ode's for heat and temperature in the atmosphere and ocean
    # --------------------------------------------------------------------
    time, x, x_drv, y = ode4u( sys , time , x_init , u , c )

    time = time / c.sec_per_yr

    T_atm = y[0:N,:]   - 273;    # deg C
    T_ocn = y[N:2*N,:] - 273;    # deg C
    RF    = y[2*N];

    # print some results
    # ----------------------------------------------------
#   print("sbc=",c.sbc, "So=", c.So, "u=",u[0])
    print("Year= ",np.round(time[-1]))
    print("The temperatures in the atmosphere layers:" )
    print(" T_atm",np.round(time[-1]),"= ",np.round(T_atm[:,-1]))
    print(" T_atm[0]",np.round(time[-1]),"= ",T_atm[0,-1])
    print("The temperatures in the    ocean    layers:" )
    print(" T_ocn",np.round(time[-1]),"= ",np.round(T_ocn[:,-1]))
    print(" T_ocn[0]",np.round(time[-1]),"= ",T_ocn[0,-1])
#   print("atm_ahc=", c.atm_ahc/1e6, "ocn_ahc=", c.ocn_ahc/1e6)

    # ---------------------------------------------------------- Plots
    fontsize = 12
    plt.rcParams['font.size'] = fontsize
    plt.ion()                 # interactive mode

#   f10 = plt.figure(10) # ---------------------------------------
#   plt.clf()
#   plt.plot(time, dt,  "-", color='Black')
#   plt.ylabel('delta-t')
#   plt.xlabel('Time (years)')
#
#   f11 = plt.figure(11) # ---------------------------------------
#   plt.clf()
#   plt.plot(time,  "-", color='Black')
#   plt.ylabel('t')
#   plt.xlabel('time index number ')
#
#   f0 = plt.figure(0) # ---------------------------------------
#   plt.clf()
#   plt.plot(time, u, "-", color='Red')
#   plt.ylabel('Short Wavelentgh Solar Input at Surface W/sq.m')
#   plt.xlabel('Time (years)')

    f1 = plt.figure(1) # ---------------------------------------
#   fig=plt.figure(figsize=(4.2,4.3),dpi=300)
    plt.clf()
#   plt.subplot(3,1,1)

    plt.plot(time, T_atm[0,:], "-", color='SkyBlue', label="atmosphere")
    plt.plot(time, T_ocn[0,:], "-", color='MidnightBlue', label="ocean")
    plt.plot(time[-1], T_atm[0,-1], "o", color='SkyBlue')
    plt.plot(time[-1], T_ocn[0,-1], "o", color='MidnightBlue')
    for k in range(1,N):
        plt.plot(time, T_atm[k,:], "--", color='SkyBlue' )
        plt.plot(time, T_ocn[k,:], "--", color='MidnightBlue')
    plt.ylabel('Temperature (°C)')
    plt.xlabel('Time (years)')
#   plt.xlim([0,200])
#   plt.ylim([0,4])
#   plt.title('Atm/ Ocean temperature anomalies given a sudden 2x CO฀ increase')
    plt.legend(loc='lower right',fontsize=fontsize-1)
    plt.grid(lw=0.25)
    plt.title("(a)",loc="left",fontsize=fontsize)

    f2 = plt.figure(2) # ---------------------------------------
#   fig=plt.figure(figsize=(4.2,4.3),dpi=300)
    plt.clf()
#   plt.subplot(3,1,1)

    plt.plot(time, y[2,:], "-", color='Orange')
    plt.ylabel('RF W/sq.m')
    plt.xlabel('Time (years)')
#   plt.xlim([0,200])
#   plt.ylim([0,4])
#   plt.title('Atm/ Ocean temperature anomalies given a sudden 2x CO฀ increase')
#   plt.legend(loc='right',fontsize=fontsize-1)
    plt.grid(lw=0.25)
    plt.title("(b)",loc="left",fontsize=fontsize)

    plt.show()

In [ ]:
run()

# Question 2

Change values of emissivity, albedo, heat transfer coefficients, number of layers, and time steps, to
 observe trends and various kinds of numerical instability.
 Describe the effects of these changes.

Answer:

Test 1: Low Albedo (More Vegetation/Dark Surface)

Represents a heavily vegetated Earth (forests, dark soil) or reduced ice cover. Dark surfaces absorb more incoming solar radiation.

In [ ]:
print("\n" + "="*70)
print("TEST 1: LOW ALBEDO - More Vegetation (aa = 0.20)")
print("="*70)

c.aa = 0.20  # Decreased from 0.35 (darker surface absorbs more solar)
run()

# Result
# Test 1 Results: Low Albedo (aa = 0.20)

# T_atm[0] = 27.28°C → +12.13°C vs baseline (15.15°C)
# T_ocn[0] = 32.48°C → +12.76°C vs baseline (19.72°C)
# T_ocn[-1] = 3°C → Unchanged

# Effect: System is much warmer - more solar absorption causes significant warming. Still warming after 15 years (not in equilibrium).

Test 2: High Albedo (Ice Age/Snow Cover)

Represents ice-covered Earth or extensive snow cover. Bright surfaces reflect more solar radiation

In [ ]:
print("\n" + "="*70)
print("TEST 2: HIGH ALBEDO - Ice Age (aa = 0.60)")
print("="*70)

c.aa = 0.60  # Increased from 0.35 (bright surface reflects more solar)
run()

#Result
# Test 2 Results: High Albedo (aa = 0.60)

# T_atm[0] = -6.68°C → -21.83°C vs baseline (15.15°C)
# T_ocn[0] = -3.12°C → -22.84°C vs baseline (19.72°C)
# T_ocn[-1] = 3°C → Unchanged

# Effect: System is much colder - less solar absorption causes dramatic cooling (glacial conditions).
# Ocean surface below freezing! Still cooling after 15 years.



Test 3: Lower Atmospheric Emissivity (Less Greenhouse Effect)

Represents atmosphere with lower greenhouse gas concentration (less CO₂, water vapor). Less effective at trapping longwave radiation.

In [ ]:
print("\n" + "="*70)
print("TEST 3: LOWER EMISSIVITY - Less GHG (ee_atm = 0.70)")
print("="*70)

c.aa = 0.35  # Reset albedo to baseline
c.ee_atm = 0.70  # Decreased from 0.78 (less greenhouse trapping)
run()

# Result
# Test 3 Results: Lower Emissivity (ee_atm = 0.70)

# T_atm[0] = 9.18°C → -5.97°C vs baseline (15.15°C)
# T_ocn[0] = 13.79°C → -5.93°C vs baseline (19.72°C)
# T_ocn[-1] = 3°C → Unchanged

# Effect: System is cooler - weaker greenhouse effect allows more heat to escape to space. Still cooling after 15 years.

Test 4: Higher Atmospheric Emissivity (More Greenhouse Effect)

Higher greenhouse gas concentration (more CO₂, water vapor). More effective at trapping longwave radiation.

In [ ]:
print("\n" + "="*70)
print("TEST 4: HIGHER EMISSIVITY - More GHG (ee_atm = 0.85)")
print("="*70)

c.ee_atm = 0.85  # Increased from 0.78 (more greenhouse trapping)
run()

# Result
# Test 4 Results: Higher Emissivity (ee_atm = 0.85)

# T_atm[0] = 23.22°C → +8.07°C vs baseline (15.15°C)
# T_ocn[0] = 27.67°C → +7.95°C vs baseline (19.72°C)
# T_ocn[-1] = 3°C → Unchanged

# Effect: System is warmer - stronger greenhouse effect traps more heat. Still warming after 15 years.



Test 5: Extreme Heat Transfer (Test Numerical Instability)

Testing what happens with unrealistically high mixing rates. Should cause numerical instability or unrealistic behavior.

In [ ]:
print("\n" + "="*70)
print("TEST 5: EXTREME HEAT TRANSFER - Testing Instability")
print("="*70)

c.ee_atm = 0.78  # Reset to baseline
c.atm_heat_transfer = 50.0  # Massively increased from 3.7
c.ocn_heat_transfer = 50.0  # Massively increased from 4.0
run()

# Result
# Test 5 Results: Extreme Heat Transfer (both = 50.0)

# T_atm[0] = -94.29°C → -109.44°C vs baseline (complete breakdown!)
# T_ocn[0] = -21.14°C → -40.86°C vs baseline
# T_ocn[-1] = 1°C

# Effect: Numerical instability observed!

# Extreme oscillations in early years (visible in plot around years 1-4)
# Unrealistic temperatures (atmosphere bottom at -94°C!)
# Non-physical behavior: the solver struggles with extremely high mixing rates
# The RK4 timestep is too large for the rapid heat transfer rates

# This demonstrates the Courant-Friedrichs-Lewy (CFL) condition:
# when diffusion coefficients are too large relative to timestep, the numerical scheme becomes unstable.

# Summary of Effects:

# Parameter Sensitivity Ranking (by temperature impact):

Albedo: ±12-23°C change (most sensitive) - directly controls solar energy input

Atmospheric emissivity: ±6-8°C change (moderate) - controls longwave radiation trapping

Heat transfer coefficients: Can cause complete instability if too extreme

# Physical Insights:

Albedo has the strongest effect because it operates on incoming solar radiation (343 W/m² globally averaged), while emissivity affects only the re-radiated longwave component

The model exhibits positive feedback potential: warming reduces ice/snow → lowers albedo → more warming

Greenhouse effect (emissivity) has substantial but more moderate impact than albedo

All non-baseline cases showed continued drift (warming or cooling), indicating the system requires longer than 15 years to reach new equilibrium

# Numerical Stability:

Extreme heat transfer coefficients (13× baseline) caused complete breakdown of numerical solution

Instability manifests as: unrealistic temperatures, oscillatory behavior, and non-physical results

Root cause: time discretization becomes too coarse relative to the physical timescales of rapid diffusion

Lesson: numerical parameters (timestep, grid resolution) must be chosen carefully relative to physical parameters to maintain stability and accuracy

# Trends Observed:

Increasing albedo → cooling (less solar absorption)

Decreasing albedo → warming (more solar absorption)

Increasing emissivity → warming (stronger greenhouse effect)

Decreasing emissivity → cooling (weaker greenhouse effect)

Extreme mixing rates → numerical instability (solver breakdown)

